In [1]:
# This produces the dataframe for WR

In [2]:
## Notes on the NFL Library ##
# the NFL python library seem to not work on Tuesday probably due to updates
# best to run this on a day other than Tues.

In [3]:
## REQUIRED ACTIONS - Include in a README doc ## 
# modify the season start date in the 'get_current_week' function
#

In [4]:
# import the libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
import glob
from IPython.display import display, HTML
from datetime import datetime
import nfl_data_py as nfl
import os
import re

In [5]:
# Set Pandas options to display all columns in a single row without wrapping
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

In [6]:
# Function to calculate the current week of the NFL season
def get_current_week():
    current_date = datetime.now()
    season_start_date = datetime(2024, 9, 4)  # Update for the season start
    current_week = ((current_date - season_start_date).days // 7) + 1
    return current_week

# Define the current NFL year, week, and season type
current_year = datetime.now().year
current_week = get_current_week()
seasontype = 2 if current_week <= 18 else 3  # Regular season or playoffs

In [7]:
# define the years to pull
# nfl.import_weekly_data(years, columns, downcast)
def get_year_range(current_year, current_week, start_year=2017):
    if current_week <= 18:  # Regular season
        return list(range(start_year, current_year + 1))
    else:  # Playoffs
        return list(range(start_year, current_year))

# Use the function
years = get_year_range(current_year, current_week)

In [8]:
# define the base columns. 
base_columns = [
    'season', 'season_type', 'week', 'player_id', 'player_name', 
    'position', 'position_group', 'recent_team'
]

In [9]:
# Import the player IDs from nfl.import_ids() - without parameters
ids_data = nfl.import_ids()

# Drop the unnecessary columns
columns_to_drop = [
    'position', 'team', 'birthdate', 'age', 'draft_year', 
    'draft_round', 'draft_pick', 'draft_ovr', 'twitter_username', 
    'height', 'weight', 'college', 'db_season'
]
ids_data = ids_data.drop(columns=columns_to_drop, errors='ignore')

# Display the resulting dataframe for review
# print(f"Columns after dropping unnecessary ones: {ids_data.columns.tolist()}")
# display(ids_data)

In [10]:
# import the weekly data from nfl.import_weekly_data(years, columns, downcast)
weekly_data = nfl.import_weekly_data(
    years=years,
    columns=base_columns
)

# display(weekly_data)

Downcasting floats.


In [11]:
## Output: a dataframe of ALL NFL athletes info and ids since 2017

# Merge the two dataframes on 'player_id' and 'gsis_id'
# Align column names for merging
ids_data = ids_data.rename(columns={'gsis_id': 'player_id'})  
id_dataframe = pd.merge(weekly_data, ids_data, on='player_id', how='inner')

# Assign the resulting dataframe to a variable
all_players_id_data = id_dataframe

# Display the resulting ID dataframe
# display(all_players_id_data)

In [12]:
## Output: a dataframe of NFL WR info and ids since 2017
# extract WR from the dataframe
# Create a new dataframe with only wide receivers
wide_receiver_ids = all_players_id_data[all_players_id_data['position'] == 'WR']

# Display the resulting dataframe for review
# display(wide_receiver_ids)

In [13]:
## TEST ##
# Define the season and week range for testing
test_season = 2022  # Replace with the desired season
week_start = 8      # Start of the range
week_end = 13        # End of the range

# Filter the dataframe for the specified season and week range
test_data = wide_receiver_ids[
    (wide_receiver_ids['season'] == test_season) &
    (wide_receiver_ids['week'] >= week_start) &
    (wide_receiver_ids['week'] <= week_end)
]

# Display the filtered dataframe
# display(test_data)

In [14]:
## TEST ##
# Compare player_id to gsis_id from import_ngs_data()

wr_id = '00-0038976'  # Replace with the desired player_id

# Filter the dataframe for the matching player_id
player_data = wide_receiver_ids[wide_receiver_ids['player_id'] == wr_id]

# Display the player's weekly data
# display(player_data)

In [15]:
## Output: a dataframe of NFL WR info, ids, and stats since 2017
# WR-specific columns (receiving-related)
wr_columns = [
    'receptions', 'targets', 'receiving_yards', 'receiving_tds',
    'receiving_fumbles', 'receiving_fumbles_lost',
    'receiving_air_yards', 'receiving_yards_after_catch',
    'receiving_first_downs', 'receiving_epa',
    'receiving_2pt_conversions', 'racr', 'target_share',
    'air_yards_share', 'wopr'
]

# Pull WR-specific columns from weekly data
wr_stats = nfl.import_weekly_data(
    years=years,
    columns=['player_id', 'season', 'week'] + wr_columns  # Include keys for merging
)

# Merge WR-specific stats with wide_receiver_ids
wr_ids_weekly_stats = pd.merge(
    wide_receiver_ids,
    wr_stats,
    on=['player_id', 'season', 'week'],  # Ensure correct alignment
    how='inner'
)

# Display the resulting dataframe for review
print(f"Shape of merged dataframe: {wr_ids_weekly_stats.shape}")
# display(wr_ids_weekly_stats.head())
display(wr_ids_weekly_stats)

Downcasting floats.
Shape of merged dataframe: (17328, 44)


,season,season_type,week,player_id,player_name,position,position_group,recent_team,mfl_id,sportradar_id,fantasypros_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr
0,2017,REG,1,00-0022921,L.Fitzgerald,WR,WR,ARI,7393,b6a61b38-5cfa-46eb-b1c5-b0255d7ebaf5,9383.0,1724.0,223.0,larryfitzgerald/2506106,5528.0,6762.0,1732.0,492934.0,FitzLa00,larry-fitzgerald-1,3730.0,1661.0,NaN,6762.0,246053.0,5571.0,NaN,Larry Fitzgerald,larry fitzgerald,6,13,74.0,0,0.0,0.0,144.0,44.0,4.0,0.997088,0,0.513889,0.276596,0.342043,0.654324
1,2017,REG,2,00-0022921,L.Fitzgerald,WR,WR,ARI,7393,b6a61b38-5cfa-46eb-b1c5-b0255d7ebaf5,9383.0,1724.0,223.0,larryfitzgerald/2506106,5528.0,6762.0,1732.0,492934.0,FitzLa00,larry-fitzgerald-1,3730.0,1661.0,NaN,6762.0,246053.0,5571.0,NaN,Larry Fitzgerald,larry fitzgerald,3,6,21.0,0,0.0,0.0,29.0,17.0,2.0,-3.455533,0,0.724138,0.166667,0.069378,0.298565
2,2017,REG,3,00-0022921,L.Fitzgerald,WR,WR,ARI,7393,b6a61b38-5cfa-46eb-b1c5-b0255d7ebaf5,9383.0,1724.0,223.0,larryfitzgerald/2506106,5528.0,6762.0,1732.0,492934.0,FitzLa00,larry-fitzgerald-1,3730.0,1661.0,NaN,6762.0,246053.0,5571.0,NaN,Larry Fitzgerald,larry fitzgerald,13,15,149.0,1,0.0,0.0,138.0,45.0,6.0,7.632769,0,1.079710,0.312500,0.369973,0.727731
3,2017,REG,4,00-0022921,L.Fitzgerald,WR,WR,ARI,7393,b6a61b38-5cfa-46eb-b1c5-b0255d7ebaf5,9383.0,1724.0,223.0,larryfitzgerald/2506106,5528.0,6762.0,1732.0,492934.0,FitzLa00,larry-fitzgerald-1,3730.0,1661.0,NaN,6762.0,246053.0,5571.0,NaN,Larry Fitzgerald,larry fitzgerald,4,7,32.0,1,0.0,0.0,31.0,18.0,1.0,0.162141,0,1.032258,0.137255,0.070938,0.255539
4,2017,REG,5,00-0022921,L.Fitzgerald,WR,WR,ARI,7393,b6a61b38-5cfa-46eb-b1c5-b0255d7ebaf5,9383.0,1724.0,223.0,larryfitzgerald/2506106,5528.0,6762.0,1732.0,492934.0,FitzLa00,larry-fitzgerald-1,3730.0,1661.0,NaN,6762.0,246053.0,5571.0,NaN,Larry Fitzgerald,larry fitzgerald,6,10,51.0,0,0.0,0.0,44.0,29.0,5.0,2.428232,0,1.159091,0.227273,0.105516,0.414770
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17323,2024,REG,2,00-0039920,M.Corley,WR,WR,NYJ,16636,bae59933-8b94-4837-990e-f0a4ced3cdbb,26023.0,NaN,11617.0,NaN,4613104.0,40944.0,NaN,3162613.0,CorlMa00,malachi-corley-1,17777.0,NaN,1607.0,40944.0,0.0,NaN,1215291.0,Malachi Corley,malachi corley,1,1,4.0,0,0.0,0.0,-1.0,5.0,0.0,-0.475780,0,0.000000,0.034483,-0.006579,0.047119
17324,2024,REG,9,00-0039920,M.Corley,WR,WR,NYJ,16636,bae59933-8b94-4837-990e-f0a4ced3cdbb,26023.0,NaN,11617.0,NaN,4613104.0,40944.0,NaN,3162613.0,CorlMa00,malachi-corley-1,17777.0,NaN,1607.0,40944.0,0.0,NaN,1215291.0,Malachi Corley,malachi corley,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,NaN,NaN
17325,2024,REG,10,00-0039920,M.Corley,WR,WR,NYJ,16636,bae59933-8b94-4837-990e-f0a4ced3cdbb,26023.0,NaN,11617.0,NaN,4613104.0,40944.0,NaN,3162613.0,CorlMa00,malachi-corley-1,17777.0,NaN,1607.0,40944.0,0.0,NaN,1215291.0,Malachi Corley,malachi corley,1,2,2.0,0,0.0,0.0,12.0,0.0,1.0,-2.246118,0,0.166667,0.060606,0.057692,0.131294
17326,2024,REG,11,00-0039920,M.Corley,WR,WR,NYJ,16636,bae59933-8b94-4837-990e-f0a4ced3cdbb,26023.0,NaN,11617.0,NaN,4613104.0,40944.0,NaN,3162613.0,CorlMa00,malachi-corley-1,17777.0,NaN,1607.0,40944.0,0.0,NaN,1215291.0,Malachi Corley,malachi corley,1,1,10.0,0,0.0,0.0,10.0,0.0,0.0,0.563583,0,1.000000,0.034483,0.080645,0.108176


In [16]:
# Output: imports the NFL next-generation stats from the nfl python library

# import the next generation stats (NGS) from nfl.import_ngs_data()
# note: ngs starts at week 0 (previous season totals) - not needed so drop those rows

# Pull NGS receiving data for the specified years
ngs_wr_df = nfl.import_ngs_data('receiving', years)

# Exclude rows where 'week' == 0 and filter for 'WR' position in one step
ngs_wr_df = ngs_wr_df[(ngs_wr_df['week'] != 0) & (ngs_wr_df['player_position'] == 'WR')]

# Drop unnecessary columns
ngs_wr_df = ngs_wr_df.drop(columns=['season_type', 'player_position', 'receptions', 'targets'], errors='ignore')

# Display the resulting dataframe
print(f"Shape of NGS WR DataFrame after dropping columns: {ngs_wr_df.shape}")
display(ngs_wr_df)

Shape of NGS WR DataFrame after dropping columns: (8215, 19)


,season,week,player_display_name,team_abbr,avg_cushion,avg_separation,avg_intended_air_yards,percent_share_of_intended_air_yards,catch_percentage,yards,rec_touchdowns,avg_yac,avg_expected_yac,avg_yac_above_expectation,player_gsis_id,player_first_name,player_last_name,player_jersey_number,player_short_name
1725,2017,1,Ryan Grant,WAS,9.936667,2.894592,4.410000,7.154639,66.666667,61.0,0,11.232500,10.072361,1.160139,00-0031068,Ryan,Grant,14,R.Grant
1726,2017,1,Martavis Bryant,PIT,8.300000,4.122054,12.688333,33.327496,33.333333,14.0,0,0.155000,4.098278,-3.943278,00-0031373,Martavis,Bryant,10,M.Bryant
1729,2017,1,Jamison Crowder,WAS,7.655000,3.177793,10.540000,19.949707,42.857143,14.0,0,1.450000,1.631897,-0.181897,00-0031941,Jamison,Crowder,80,J.Crowder
1732,2017,1,Nelson Agholor,PHI,7.423750,2.462620,10.463750,20.274656,75.000000,86.0,1,5.611667,3.262470,2.349197,00-0031549,Nelson,Agholor,13,N.Agholor
1733,2017,1,John Brown,ARI,7.360000,2.751526,13.422222,28.208481,44.444444,32.0,0,-0.377500,0.961993,-1.339493,00-0031051,John,Brown,12,J.Brown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13276,2024,19,Ladd McConkey,LAC,8.009231,2.820511,13.993571,50.646295,64.285714,197.0,1,9.031111,1.758951,7.272160,00-0039915,Andrew,McConkey,15,L.McConkey
13277,2024,19,John Metchie III,HOU,7.250000,3.162210,9.848750,25.264542,50.000000,28.0,0,1.392500,1.775827,-0.383327,00-0037614,John,Metchie,8,J.Metchie
13278,2024,19,Nico Collins,HOU,6.756250,2.489115,14.006250,35.929584,87.500000,122.0,1,8.638571,5.577739,3.060833,00-0036554,Nico,Collins,12,N.Collins
13280,2024,19,George Pickens,PIT,6.202000,2.788453,12.334000,40.312459,100.000000,87.0,1,5.066000,4.451983,0.614017,00-0037247,George,Pickens,14,G.Pickens


In [17]:
# Ouput: a dataframe of NFL WR info, ids, weekly stats, and next-gen stats since 2017


# Joinswr_ids_weekly_stats dataframe with ngs_wr_df using the keys 'player_id' and 'player_gsis_id'
# Merge wr_ids_weekly_stats with ngs_wr_df using a left join
wr_ids_ngs_weekly_stats = pd.merge(
    wr_ids_weekly_stats,
    ngs_wr_df,
    left_on=['player_id', 'season', 'week'],  # Keys from wr_ids_weekly_stats
    right_on=['player_gsis_id', 'season', 'week'],  # Keys from ngs_wr_df
    how='left'  # Retain all rows from wr_ids_weekly_stats
)

# Display the shape of the resulting dataframe
print(f"Shape of merged dataframe: {wr_ids_ngs_weekly_stats.shape}")
print(f"Row count matches: {wr_ids_weekly_stats.shape[0] == wr_ids_ngs_weekly_stats.shape[0]}")


# Display a sample of the merged dataframe
display(wr_ids_ngs_weekly_stats)

Shape of merged dataframe: (17328, 61)
Row count matches: True


,season,season_type,week,player_id,player_name,position,position_group,recent_team,mfl_id,sportradar_id,fantasypros_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,player_display_name,team_abbr,avg_cushion,avg_separation,avg_intended_air_yards,percent_share_of_intended_air_yards,catch_percentage,yards,rec_touchdowns,avg_yac,avg_expected_yac,avg_yac_above_expectation,player_gsis_id,player_first_name,player_last_name,player_jersey_number,player_short_name
0,2017,REG,1,00-0022921,L.Fitzgerald,WR,WR,ARI,7393,b6a61b38-5cfa-46eb-b1c5-b0255d7ebaf5,9383.0,1724.0,223.0,larryfitzgerald/2506106,5528.0,6762.0,1732.0,492934.0,FitzLa00,larry-fitzgerald-1,3730.0,1661.0,NaN,6762.0,246053.0,5571.0,NaN,Larry Fitzgerald,larry fitzgerald,6,13,74.0,0,0.0,0.0,144.0,44.0,4.0,0.997088,0,0.513889,0.276596,0.342043,0.654324,Larry Fitzgerald,ARI,5.936667,2.293974,10.764615,32.677938,46.153846,74.0,0.0,7.375000,8.630824,-1.255824,00-0022921,Larry,Fitzgerald,11.0,L.Fitzgerald
1,2017,REG,2,00-0022921,L.Fitzgerald,WR,WR,ARI,7393,b6a61b38-5cfa-46eb-b1c5-b0255d7ebaf5,9383.0,1724.0,223.0,larryfitzgerald/2506106,5528.0,6762.0,1732.0,492934.0,FitzLa00,larry-fitzgerald-1,3730.0,1661.0,NaN,6762.0,246053.0,5571.0,NaN,Larry Fitzgerald,larry fitzgerald,3,6,21.0,0,0.0,0.0,29.0,17.0,2.0,-3.455533,0,0.724138,0.166667,0.069378,0.298565,Larry Fitzgerald,ARI,4.746667,2.808189,5.010000,7.075605,50.000000,21.0,0.0,5.873333,6.784866,-0.911533,00-0022921,Larry,Fitzgerald,11.0,L.Fitzgerald
2,2017,REG,3,00-0022921,L.Fitzgerald,WR,WR,ARI,7393,b6a61b38-5cfa-46eb-b1c5-b0255d7ebaf5,9383.0,1724.0,223.0,larryfitzgerald/2506106,5528.0,6762.0,1732.0,492934.0,FitzLa00,larry-fitzgerald-1,3730.0,1661.0,NaN,6762.0,246053.0,5571.0,NaN,Larry Fitzgerald,larry fitzgerald,13,15,149.0,1,0.0,0.0,138.0,45.0,6.0,7.632769,0,1.079710,0.312500,0.369973,0.727731,Larry Fitzgerald,ARI,6.556000,3.289943,9.675333,33.997048,86.666667,149.0,1.0,3.536154,3.383512,0.152642,00-0022921,Larry,Fitzgerald,11.0,L.Fitzgerald
3,2017,REG,4,00-0022921,L.Fitzgerald,WR,WR,ARI,7393,b6a61b38-5cfa-46eb-b1c5-b0255d7ebaf5,9383.0,1724.0,223.0,larryfitzgerald/2506106,5528.0,6762.0,1732.0,492934.0,FitzLa00,larry-fitzgerald-1,3730.0,1661.0,NaN,6762.0,246053.0,5571.0,NaN,Larry Fitzgerald,larry fitzgerald,4,7,32.0,1,0.0,0.0,31.0,18.0,1.0,0.162141,0,1.032258,0.137255,0.070938,0.255539,Larry Fitzgerald,ARI,8.400000,2.609960,5.661429,8.939165,57.142857,32.0,1.0,4.125000,3.548166,0.576834,00-0022921,Larry,Fitzgerald,11.0,L.Fitzgerald
4,2017,REG,5,00-0022921,L.Fitzgerald,WR,WR,ARI,7393,b6a61b38-5cfa-46eb-b1c5-b0255d7ebaf5,9383.0,1724.0,223.0,larryfitzgerald/2506106,5528.0,6762.0,1732.0,492934.0,FitzLa00,larry-fitzgerald-1,3730.0,1661.0,NaN,6762.0,246053.0,5571.0,NaN,Larry Fitzgerald,larry fitzgerald,6,10,51.0,0,0.0,0.0,44.0,29.0,5.0,2.428232,0,1.159091,0.227273,0.105516,0.414770,Larry Fitzgerald,ARI,6.392222,2.449503,4.217000,11.136346,60.000000,51.0,0.0,6.521667,5.652730,0.868937,00-0022921,Larry,Fitzgerald,11.0,L.Fitzgerald
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17323,2024,REG,2,00-0039920,M.Corley,WR,WR,NYJ,16636,bae59933-8b94-4837-990e-f0a4ced3cdbb,26023.0,NaN,11617.0,NaN,4613104.0,40944.0,NaN,3162613.0,CorlMa00,malachi-corley-1,17777.0,NaN,1607.0,40944.0,0.0,NaN,1215291.0,Malachi Corley,malachi corley,1,1,4.0,0,0.0,0.0,-1.0,5.0,0.0,-0.475780,0,0.000000,0.034483,-0.006579,0.047119,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17324,2024,REG,9

In [18]:
## TEST ##
# Define the season and week range for testing
test_season = 2024  # Replace with the desired season
week_start = 1      # Start of the range
week_end = 18       # End of the range

# Filter the merged dataframe for the specified season and week range
test_data = wr_ids_ngs_weekly_stats[
    (wr_ids_ngs_weekly_stats['season'] == test_season) &
    (wr_ids_ngs_weekly_stats['week'] >= week_start) &
    (wr_ids_ngs_weekly_stats['week'] <= week_end)
]

# Display the filtered dataframe for testing
print(f"Filtered data for season {test_season}, weeks {week_start}-{week_end}:")
display(test_data)

Filtered data for season 2024, weeks 1-18:


,season,season_type,week,player_id,player_name,position,position_group,recent_team,mfl_id,sportradar_id,fantasypros_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,player_display_name,team_abbr,avg_cushion,avg_separation,avg_intended_air_yards,percent_share_of_intended_air_yards,catch_percentage,yards,rec_touchdowns,avg_yac,avg_expected_yac,avg_yac_above_expectation,player_gsis_id,player_first_name,player_last_name,player_jersey_number,player_short_name
15202,2024,REG,1,00-0030035,A.Thielen,WR,WR,CAR,11938,2fa2b2da-4aa9-44b5-b27e-56876dfe2ad4,13429.0,8288.0,1689.0,NaN,16460.0,27277.0,NaN,2059362.0,ThieAd00,NaN,8986.0,9054.0,308.0,27277.0,733643.0,15534.0,733643.0,Adam Thielen,adam thielen,3,4,49.0,0,0.0,0.0,64.0,18.0,2.0,4.642717,0,0.765625,0.137931,0.181818,0.334169,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15203,2024,REG,2,00-0030035,A.Thielen,WR,WR,CAR,11938,2fa2b2da-4aa9-44b5-b27e-56876dfe2ad4,13429.0,8288.0,1689.0,NaN,16460.0,27277.0,NaN,2059362.0,ThieAd00,NaN,8986.0,9054.0,308.0,27277.0,733643.0,15534.0,733643.0,Adam Thielen,adam thielen,2,3,20.0,0,0.0,0.0,18.0,2.0,1.0,-2.458903,0,1.111111,0.115385,0.233766,0.336713,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15204,2024,REG,3,00-0030035,A.Thielen,WR,WR,CAR,11938,2fa2b2da-4aa9-44b5-b27e-56876dfe2ad4,13429.0,8288.0,1689.0,NaN,16460.0,27277.0,NaN,2059362.0,ThieAd00,NaN,8986.0,9054.0,308.0,27277.0,733643.0,15534.0,733643.0,Adam Thielen,adam thielen,3,5,40.0,1,0.0,0.0,44.0,5.0,2.0,3.056416,0,0.909091,0.135135,0.205607,0.346628,Adam Thielen,CAR,5.504000,3.307096,8.512,18.930700,60.0,40.0,1.0,2.806667,1.664967,1.141699,00-0030035,Adam,Thielen,19.0,A.Thielen
15205,2024,REG,12,00-0030035,A.Thielen,WR,WR,CAR,11938,2fa2b2da-4aa9-44b5-b27e-56876dfe2ad4,13429.0,8288.0,1689.0,NaN,16460.0,27277.0,NaN,2059362.0,ThieAd00,NaN,8986.0,9054.0,308.0,27277.0,733643.0,15534.0,733643.0,Adam Thielen,adam thielen,3,4,57.0,0,0.0,0.0,34.0,35.0,2.0,4.730047,0,1.676471,0.121212,0.116838,0.263605,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15206,2024,REG,13,00-0030035,A.Thielen,WR,WR,CAR,11938,2fa2b2da-4aa9-44b5-b27e-56876dfe2ad4,13429.0,8288.0,1689.0,NaN,16460.0,27277.0,NaN,2059362.0,ThieAd00,NaN,8986.0,9054.0,308.0,27277.0,733643.0,15534.0,733643.0,Adam Thielen,adam thielen,8,10,99.0,1,0.0,0.0,112.0,23.0,4.0,6.234365,0,0.883929,0.243902,0.279302,0.561365,Adam Thielen,CAR,5.335556,3.835267,10.644,26.598696,80.0,99.0,1.0,3.440000,3.374431,0.065569,00-0030035,Adam,Thielen,19.0,A.Thielen
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17323,2024,REG,2,00-0039920,M.Corley,WR,WR,NYJ,16636,bae59933-8b94-4837-990e-f0a4ced3cdbb,26023.0,NaN,11617.0,NaN,4613104.0,40944.0,NaN,3162613.0,CorlMa00,malachi-corley-1,17777.0,NaN,1607.0,40944.0,0.0,NaN,1215291.0,Malachi Corley,malachi corley,1,1,4.0,0,0.0,0.0,-1.0,5.0,0.0,-0.475780,0,0.000000,0.034483,-0.006579,0.047119,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17324,2024,REG,9,00-0039920,M.Corley,WR,WR,NYJ,16636,bae59933-8b94-4837-990e-f0a4ced3cdbb,26023.0,NaN,11617.0,NaN,4613104.0,40944.0,NaN,3162613.0,CorlMa00,malachi-corley-1,17777.0,NaN,1607.0,40944.0,0.0,NaN,1215291.0,Malachi Corley,malachi corley,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17325,2024,REG,10,00-0039920,M.Corley,WR,WR,NYJ,16636,bae59933-8b94-4837-990e-f0a4ced3cdb

In [19]:
## TEST ##
# Compare player_id to player_gsis_id from the merged dataframe
wr_id = '00-0038976'  # Replace with the desired player_id

# Filter the merged dataframe for the matching player_id
player_data = wr_ids_ngs_weekly_stats[wr_ids_ngs_weekly_stats['player_id'] == wr_id]

# Display the player's weekly data for testing
print(f"Weekly data for player_id {wr_id}:")
display(player_data)

Weekly data for player_id 00-0038976:


,season,season_type,week,player_id,player_name,position,position_group,recent_team,mfl_id,sportradar_id,fantasypros_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,player_display_name,team_abbr,avg_cushion,avg_separation,avg_intended_air_yards,percent_share_of_intended_air_yards,catch_percentage,yards,rec_touchdowns,avg_yac,avg_expected_yac,avg_yac_above_expectation,player_gsis_id,player_first_name,player_last_name,player_jersey_number,player_short_name
15024,2023,REG,1,00-0038976,M.Mims,WR,WR,DEN,16189,fcd780e6-6d43-4c2e-b740-59ae09911345,23080.0,NaN,9494.0,NaN,4686472.0,40090.0,NaN,3162586.0,MimsMa00,marvin-mims-1,16849.0,NaN,1442.0,40090.0,0.0,NaN,1213799.0,Marvin Mims,marvin mims,2,2,9.0,0,0.0,0.0,3.0,6.0,1.0,-1.011986,0,3.000000,0.062500,0.022727,0.109659,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15025,2023,REG,2,00-0038976,M.Mims,WR,WR,DEN,16189,fcd780e6-6d43-4c2e-b740-59ae09911345,23080.0,NaN,9494.0,NaN,4686472.0,40090.0,NaN,3162586.0,MimsMa00,marvin-mims-1,16849.0,NaN,1442.0,40090.0,0.0,NaN,1213799.0,Marvin Mims,marvin mims,2,2,113.0,1,0.0,0.0,91.0,22.0,2.0,9.449176,0,1.241758,0.074074,0.320423,0.335407,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15026,2023,REG,3,00-0038976,M.Mims,WR,WR,DEN,16189,fcd780e6-6d43-4c2e-b740-59ae09911345,23080.0,NaN,9494.0,NaN,4686472.0,40090.0,NaN,3162586.0,MimsMa00,marvin-mims-1,16849.0,NaN,1442.0,40090.0,0.0,NaN,1213799.0,Marvin Mims,marvin mims,3,5,73.0,0,0.0,0.0,127.0,22.0,2.0,3.862901,0,0.574803,0.138889,0.323155,0.434542,Marvin Mims,DEN,7.330000,3.212112,23.070,28.978772,60.0,73.0,0.0,8.380000,8.195010,0.184990,00-0038976,Marvin,Mims,19.0,M.Mims
15027,2023,REG,4,00-0038976,M.Mims,WR,WR,DEN,16189,fcd780e6-6d43-4c2e-b740-59ae09911345,23080.0,NaN,9494.0,NaN,4686472.0,40090.0,NaN,3162586.0,MimsMa00,marvin-mims-1,16849.0,NaN,1442.0,40090.0,0.0,NaN,1213799.0,Marvin Mims,marvin mims,2,2,47.0,0,0.0,0.0,29.0,18.0,1.0,1.514018,0,1.620690,0.076923,0.175758,0.238415,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15028,2023,REG,5,00-0038976,M.Mims,WR,WR,DEN,16189,fcd780e6-6d43-4c2e-b740-59ae09911345,23080.0,NaN,9494.0,NaN,4686472.0,40090.0,NaN,3162586.0,MimsMa00,marvin-mims-1,16849.0,NaN,1442.0,40090.0,0.0,NaN,1213799.0,Marvin Mims,marvin mims,1,1,4.0,0,0.0,0.0,-4.0,8.0,0.0,-0.853424,0,0.000000,0.037037,-0.030303,0.034343,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15029,2023,REG,7,00-0038976,M.Mims,WR,WR,DEN,16189,fcd780e6-6d43-4c2e-b740-59ae09911345,23080.0,NaN,9494.0,NaN,4686472.0,40090.0,NaN,3162586.0,MimsMa00,marvin-mims-1,16849.0,NaN,1442.0,40090.0,0.0,NaN,1213799.0,Marvin Mims,marvin mims,0,1,0.0,0,0.0,0.0,24.0,0.0,0.0,-0.629632,0,0.000000,0.041667,0.155844,0.171591,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15030,2023,REG,8,00-0038976,M.Mims,WR,WR,DEN,16189,fcd780e6-6d43-4c2e-b740-59ae09911345,23080.0,NaN,9494.0,NaN,4686472.0,40090.0,NaN,3162586.0,MimsMa00,marvin-mims-1,16849.0,NaN,1442.0,40090.0,0.0,NaN,1213799.0,Marvin Mims,marvin mims,1,1,0.0,0,0.0,0.0,-3.0,3.0,0.0,-0.230882,0,0.000000,0.055556,-0.024390,0.066260,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15031,2023,REG,10,00-0038976,M.Mims,WR,WR,DEN,16189,fcd780e6-6d43-4c2e-b740-59ae09911345,23080.0,NaN,9494.0,NaN,4686472.0,40090.0,NaN,3162586.0,MimsMa00,marvin-mims-1,16849.0,NaN,1442.0,40090.0,0.0,NaN,1213799.0,Marvin Mims,marvin mims,0,1,0.0,0,0.0,0.0,-6.0,0.0,0.0,-0.848008,0,0.000000,0.034483,-0.060000,0.009724,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15032,2023,REG,11,00-0038976,M.Mims,WR,WR,DEN,

In [ ]:
## Next Task ##
# import the pfr data
# merge with the wr_ids_ngs_weekly_stats

In [72]:
# output: pro-football reference dataframe that contains WR data from the python nfl library
pfr_rec_df = nfl.import_weekly_pfr('rec', [2024])
pfr_rec_df.to_csv('pfr_rec_stats.csv', index=False)
pfr_rec_df

,game_id,pfr_game_id,season,week,game_type,team,opponent,pfr_player_name,pfr_player_id,rushing_broken_tackles,receiving_broken_tackles,passing_drops,passing_drop_pct,receiving_drop,receiving_drop_pct,receiving_int,receiving_rat
0,2024_01_BAL_KC,202409050kan,2024,1,REG,KC,BAL,Rashee Rice,RiceRa01,NaN,1.0,NaN,NaN,0.0,0.000,1.0,74.8
1,2024_01_BAL_KC,202409050kan,2024,1,REG,KC,BAL,Noah Gray,GrayNo00,NaN,0.0,NaN,NaN,0.0,0.000,0.0,118.1
2,2024_01_BAL_KC,202409050kan,2024,1,REG,KC,BAL,Travis Kelce,KelcTr00,NaN,0.0,NaN,NaN,1.0,0.250,0.0,100.0
3,2024_01_BAL_KC,202409050kan,2024,1,REG,KC,BAL,Isiah Pacheco,PachIs00,NaN,1.0,NaN,NaN,1.0,0.333,0.0,103.5
4,2024_01_BAL_KC,202409050kan,2024,1,REG,KC,BAL,Xavier Worthy,WortXa00,NaN,0.0,NaN,NaN,0.0,0.000,0.0,149.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4249,2024_18_MIN_DET,202501050det,2024,18,REG,MIN,DET,Jalen Nailor,NailJa00,NaN,0.0,NaN,NaN,0.0,0.000,0.0,116.7
4250,2024_18_MIN_DET,202501050det,2024,18,REG,MIN,DET,Cam Akers,AkerCa00,NaN,0.0,NaN,NaN,0.0,0.000,0.0,79.2
4251,2024_18_MIN_DET,202501050det,2024,18,REG,MIN,DET,T.J. Hockenson,HockTJ00,NaN,0.0,NaN,NaN,0.0,0.000,0.0,39.6
4252,2024_18_MIN_DET,202501050det,2024,18,REG,MIN,DET,Josh Oliver,OlivJo00,NaN,0.0,NaN,NaN,0.0,0.000,0.0,79.9
